In [ ]:
import pandas as pd
import numpy as np
# 시각화 한글폰트 설정
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as spst
from statsmodels.graphics.mosaicplot import mosaic

plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",#"NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

In [ ]:
def num_num(x, y, data):
    
    plt.figure(figsize=(12,8))
    sns.scatterplot(x=x, y=y, data=data)
    plt.grid()
    plt.show()

    result = spst.pearsonr(data[x], data[y])
    print(f'상관계수 : {result[0]}')
    print(f'p_value : {result[1]}')

    flag = '양'
    if result[1] < 0.05:
        if result[0] < 0:
            flag = '음'

        if abs(result[0]) >= 0.5:
            print(f'{x}와 {y}는 {flag}의 강한 상관관계를 가진다')
        elif abs(result[0]) >= 0.3:
            print(f'{x}와 {y}는 {flag}의 적당한 상관관계를 가진다')
        else:
            print(f'{x}와 {y}는 {flag}의 약한 상관관계를 가진다')


def num_cat(x, y, data):
    plt.subplot(2,1,1)
    sns.kdeplot(x=x, data=data, hue = y, common_norm=False)
    plt.grid()
    plt.subplot(2,1,2)
    sns.kdeplot(x=x, data=data, hue = y, multiple = 'fill')
    plt.grid()
    plt.tight_layout()
    plt.show()


def cat_num(x, y, data):
    sns.barplot(x=x, y=y, data=data)
    plt.grid()
    plt.show()

    categories = [cat for cat in data[x].unique()]

    lst = []
    for value in categories:
        lst.append(data.loc[data[x]== value, y])
    
    if len(categories) == 2:
        print('T 검정 결과는')
        result = spst.ttest_ind(*lst)
        
        if result[1] > 0.05:
            print(f'p-value가 0.05를 초과합니다 p-value : {result[1]}')
            print('범주별 평균이 차이가 없습니다')
        else:
            print(f't 통계량 값은 : {result[0]}')
            print(f'p-vaule 값은 : {result[1]}')
        
    elif len(categories) > 2:
        print('ANOVA 결과는')
        #display(spst.f_oneway(*lst))
        result = spst.f_oneway(*lst)
        if result[1] > 0.05:
            print(f'p-value가 0.05를 초과합니다 p-value : {result[1]}')
            print('범주별 평균이 차이가 없습니다')
        else:
            print(f'F 통계량 값은 : {result[0]}')
            print(f'p-vaule 값은 : {result[1]}')
    else:
        print('오류발생, 범주의 수가 2개 미만입니다.')

def cat_cat(x, y, data):
    pd.crosstab(data[x], data[y], normalize='all')
    mosaic(data, [x, y])
    plt.axhline(data[y].mean(), color = 'r')
    plt.grid()
    plt.show()

    table = pd.crosstab(data[x], data[y])
    result = spst.chi2_contingency(table)
    print('카이제곱통계량', result[0])
    print('p-value', result[1])
    print('기대빈도\n',result[3])

In [ ]:
# 단변량 분석

def eda1_n(var, data, bins=20):

    display(data[[var]].describe().T)

    plt.figure(figsize=(15,10))

    plt.subplot(2,1,1)
    sns.histplot(x=var, data=data, bins=bins, kde = True)

    plt.subplot(2,1,2)
    sns.boxplot(x = var, data = data)
    plt.grid()
    plt.show

def eda1_c(var, data):
    sns.countplot(x=var, data=data)
    plt.grid()
    plt.show()

In [ ]:
def num_uniV(x, data, bins=20): # 숫자형 단변량, x - 변수, data - 데이터, bins - 구간의 갯수 (optional)
    plt.figure(figsize=(10,5))
    plt.subplot(2,1,1)
    plt.hist(data[x], bins = bins, edgecolor = 'gray') 
    plt.xlabel(x)
    plt.ylabel('Frequency')
    plt.grid()

    plt.subplot(2,1,2)
    sns.kdeplot(data[x])
    plt.grid()
    plt.show()
    
    plt.figure(figsize=(12,8))
    plt.subplot(2,1,1)
    sns.histplot(data[x], kde=True)
    plt.xlabel(x)
    plt.ylabel('Frequency')
    plt.grid()
    
    plt.subplot(2,1,2)
    plt.boxplot(data[x])
    plt.grid()
    plt.xlabel(x)
    plt.tight_layout()
    plt.show()

def cat_uniV(x, data): # 범주형 단변량 x = 변수, data = 데이터
    sns.countplot(x=x, data=data)
    plt.grid()
    plt.show()

In [ ]:
## 시계열 데이터가 있을 때, 결측치를 선형적으로 채워주는 code, 

#시계열 데이터가 따로 없을 경우, 열 값에 따라 선형적으로 채움.
df = df.interpolate() # method='linear' 가 기본값

#시계열 데이터가 있을 경우, 시계열에 따라 선형적으로 채움.
df['date'] = pd.to_datetime(df['date']) # 시계열 데이터로 변환
df.set_index('date', inplace=True) # 시계열 데이터를 인덱스로 변환
df = df.interpolate(method='time')
df.reset_index(inplace=True) # 시계열 데이터와 인덱스 분리

#pytimekr을 쓰면 공휴일을 알아낼 수 있다.